In [84]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import metrics, preprocessing, model_selection
from functools import reduce
pd.options.display.max_columns = 100

In [85]:
train_df = pd.read_csv("../train_data/train.csv")
test_df = pd.read_csv("../test_data/test.csv")
print(train_df.shape, test_df.shape)

(341424, 24) (146765, 23)


In [86]:
train_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346


In [87]:
def date_col_derivation(df, col_list, date_format="%d/%m/%y"):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format=date_format)
        df[col + "_in_seconds"] = (df[col] - dt.datetime(1970,1,1)).dt.total_seconds()
        df[col + "_month"] = df[col].dt.month
        df[col + "_year"] = df[col].dt.year
        df[col + "_week"] = df[col].dt.year
    return df

date_cols = ["booking_date", "checkin_date", "checkout_date"]
train_df= date_col_derivation(df=train_df, col_list=date_cols, date_format="%d/%m/%y")
test_df= date_col_derivation(df=test_df, col_list=date_cols, date_format="%d/%m/%y")

In [88]:
def date_diff(df, date_col1,date_col2, diff_format="days"):
    return (df[date_col1] - df[date_col2]).dt.days

train_df["days_stay"] = date_diff(train_df,"checkout_date","checkin_date")
test_df["days_stay"] = date_diff(test_df,"checkout_date","checkin_date")
train_df["days_advance_booking"] = date_diff(train_df,"checkin_date","booking_date")
test_df["days_advance_booking"] = date_diff(test_df,"checkin_date","booking_date")

In [89]:
train_df = train_df.drop("amount_spent_per_room_night_scaled", axis=1)

In [90]:
all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
all_df = all_df.sort_values(by="checkin_date").reset_index(drop=True)
print(all_df.shape)

(488189, 37)


In [91]:
additional_cols = ["memberid", "resort_id", "state_code_residence", "checkin_date", "booking_date"]
new_df = all_df[["reservation_id"] + additional_cols] 

In [92]:
new_df.columns

Index(['reservation_id', 'memberid', 'resort_id', 'state_code_residence',
       'checkin_date', 'booking_date'],
      dtype='object')

In [93]:
# Function to group , aggregate multiple columms by any(pandas supported) method
def agg_col(df, groupy_col, agg_col_list, aggrigate_method):
    new_df = None
    for agg_col in agg_col_list:
        
        gdf = getattr(df.groupby(groupy_col)[agg_col],aggrigate_method)().reset_index()
        gdf.columns = [groupy_col , "member_"+agg_col+"_"+aggrigate_method]
        if new_df is None:
            new_df = gdf
        else:
            new_df = pd.merge(new_df, gdf, on="memberid", how="left")
    
    return new_df

In [94]:

### Aggregate columns by mean
cols_mean = ["booking_date_in_seconds", "checkin_date_in_seconds", "days_stay", "days_advance_booking", "roomnights"]
df_mean= agg_col(df=all_df, groupy_col="memberid", agg_col_list=cols_mean,aggrigate_method= "mean")

### Aggregate columns by sum
cols_sum= ["days_stay", "roomnights"]
df_sum = agg_col(df=all_df, groupy_col="memberid", agg_col_list=cols_sum,aggrigate_method= "sum")

### Aggregate columns by count of unique values
cols_nunique=  ["resort_id"]
df_nunique = agg_col(df=all_df, groupy_col="memberid", agg_col_list=cols_nunique,aggrigate_method= "nunique")

In [95]:
# Merge all dfs
dfs_list = [new_df, df_mean, df_sum, df_nunique]

new_df = reduce(lambda left,right: pd.merge(left,right,on='memberid',how="left"), dfs_list)

In [96]:
list(new_df.columns)

['reservation_id',
 'memberid',
 'resort_id',
 'state_code_residence',
 'checkin_date',
 'booking_date',
 'member_booking_date_in_seconds_mean',
 'member_checkin_date_in_seconds_mean',
 'member_days_stay_mean',
 'member_days_advance_booking_mean',
 'member_roomnights_mean',
 'member_days_stay_sum',
 'member_roomnights_sum',
 'member_resort_id_nunique']

In [67]:
def groupby_time_delta(df, groupby_col, time_col, shift_by):
    
    if shift_by >0:
        shift_type= "next"
    else:
        shift_type= "prev"
    
    shift_col_name = shift_type+"_"+time_col
    df[shift_col_name] = df.groupby(groupby_col)[time_col].shift(shift_by)
    
    new_delta_col = "time_gap_"+time_col+"_"+shift_type+"_"+ str(abs(shift_by))
    
    df[new_delta_col] = df["booking_date_in_seconds"] - df["prev_booking_date"]
    
    return df, shift_col_name, new_delta_col

In [121]:
test_df= all_df
# Create a list of columns and number of perviou/next events to look for each event.

time_shift_cols = [["booking_date_in_seconds",1],
                  ["booking_date_in_seconds",-1],
                  ["checkin_date_in_seconds",1],
                  ["checkin_date_in_seconds",-1],
                  ["checkout_date_in_seconds",1],
                   ["checkout_date_in_seconds",-1],
                   ["days_stay",1],
                   ["days_stay",-1],
                   ["roomnights",1],
                   ["roomnights",-1], 
                   ["days_advance_booking",1],
                   ["days_advance_booking",-1]               
                   ]
group_by_col= "memberid"

new_shift_cols=[]
new_delta_cols=[]

# Iterate among each column, shift and extract time deltas
for col, shift in time_shift_cols:
    all_df,shfit_col,delta_col =  groupby_time_delta(all_df, groupby_col= group_by_col, time_col=col, shift_by=shift)
    new_shift_cols.append(shfit_col)
    new_delta_cols.append(delta_col)

# Join all new columns to the new_df
new_df = pd.merge(new_df,all_df[["reservation_id"] + new_delta_cols],on="reservation_id")

In [119]:
list(new_df.columns)

['reservation_id',
 'memberid',
 'resort_id',
 'state_code_residence',
 'checkin_date',
 'booking_date',
 'member_booking_date_in_seconds_mean',
 'member_checkin_date_in_seconds_mean',
 'member_days_stay_mean',
 'member_days_advance_booking_mean',
 'member_roomnights_mean',
 'member_days_stay_sum',
 'member_roomnights_sum',
 'member_resort_id_nunique',
 'member_cumnum_of_booking',
 'cumsum_member_days_stay',
 'cumsum_member_total_pax',
 'time_gap_booking_prev',
 'time_gap_booking_next',
 'time_gap_checkin_prev',
 'time_gap_checkin_next',
 'time_gap_checkout_prev',
 'time_gap_checkout_next',
 'time_gap_booking_date_in_seconds_next_1',
 'time_gap_booking_date_in_seconds_prev_1',
 'time_gap_checkin_date_in_seconds_next_1',
 'time_gap_checkin_date_in_seconds_prev_1',
 'time_gap_checkout_date_in_seconds_next_1',
 'time_gap_checkout_date_in_seconds_prev_1']

In [ ]:
### Info on prev and next visits (no change from old code)
for col in ["channel_code", "room_type_booked_code", "resort_type_code", "main_product_code"]:
    all_df["prev_"+col] = all_df.groupby("memberid")[col].shift(1)
    new_df["prev_diff_"+col] = (all_df[col] == all_df["prev_"+col]).astype(int)
    
    all_df["next_"+col] = all_df.groupby("memberid")[col].shift(-1)
    new_df["next_diff_"+col] = (all_df[col] == all_df["next_"+col]).astype(int)

In [124]:
### pivot on member and resort ( no change from old code)
gdf = pd.pivot_table(all_df, index="memberid", columns="resort_id", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

gdf = pd.pivot_table(all_df, index="memberid", columns="checkin_date_year", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

gdf = pd.pivot_table(all_df, index="memberid", columns="resort_type_code", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

gdf = pd.pivot_table(all_df, index="memberid", columns="room_type_booked_code", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

In [125]:
list(new_df.columns)

['reservation_id',
 'memberid',
 'resort_id',
 'state_code_residence',
 'checkin_date',
 'booking_date',
 'member_booking_date_in_seconds_mean',
 'member_checkin_date_in_seconds_mean',
 'member_days_stay_mean',
 'member_days_advance_booking_mean',
 'member_roomnights_mean',
 'member_days_stay_sum',
 'member_roomnights_sum',
 'member_resort_id_nunique',
 'member_cumnum_of_booking',
 'cumsum_member_days_stay',
 'cumsum_member_total_pax',
 'time_gap_booking_prev',
 'time_gap_booking_next',
 'time_gap_checkin_prev',
 'time_gap_checkin_next',
 'time_gap_checkout_prev',
 'time_gap_checkout_next',
 'time_gap_booking_date_in_seconds_next_1_x',
 'time_gap_booking_date_in_seconds_prev_1_x',
 'time_gap_checkin_date_in_seconds_next_1_x',
 'time_gap_checkin_date_in_seconds_prev_1_x',
 'time_gap_checkout_date_in_seconds_next_1_x',
 'time_gap_checkout_date_in_seconds_prev_1_x',
 'time_gap_booking_date_in_seconds_next_1_y',
 'time_gap_booking_date_in_seconds_prev_1_y',
 'time_gap_checkin_date_in_secon

In [126]:
# End of notebook